In [1]:
import numpy as np

In [ ]:
def length(vector, axis=1):
    return np.linalg.norm(vector, ord=None, axis=axis)

def avgfilter(signal, npoints):
    if len(signal.shape) is 1:
        shape = (npoints, )
    else:
        shape = (npoints, signal.shape[1])

    coef = 1. / float(npoints)
    transients = np.zeros(shape)
    vals = np.concatenate((transients, signal), axis=0)    
    end = vals.shape[0]

    y = np.zeros_like(signal)
    for p in range(npoints):
        y = y + vals[npoints-p:end-p]*coef

    return y

In [2]:
class Gesture(object):
    
    def __init__(self):
        pass

    
class LabanApproximator(object):
    
    def __init__(self, lma_segment):
        self.frameRate = lma_segment.meta_data['frameRate']
        self.joint_keys = lma_segment.joint_keys
        self.joint_positions = lma_segment.joint_data
        self.joint_thresholds = lma_segment.segmentation_thresholds
        
    def __joint_velocities(self):
        return []
        
    def __joint_accelerations(self):
        return []
        
    
        
        
        
        
        
        